In [2]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from bertopic import BERTopic
from konlpy.tag import Okt, Komoran, Kkma, Mecab
import konlpy.jvm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
import re
import os
import sentencepiece as spm
import html
import requests
import json

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
def get_standard_dictionary(word):
    url = "https://stdict.korean.go.kr/api/search.do"
    params = {
        "key": "BCF1FAA297693D9DBF6299ACCBDD97FF", # 국립국어원에서 API키를 발급받아서 입력하세요
        "q": word,
        "req_type": "json",
    }
    response = requests.get(url, params=params)
    response.json(encoding = 'UTF-8')
    return response

In [4]:
path_dir = './'
data = pd.read_feather('./dataset.ftr')
data['timestamp'] = data['date']
#data = data[['timestamp','corrected_twit']]
#data = data.drop_duplicates()


In [5]:
data = data.drop_duplicates(subset=['twit'])

In [6]:
data = data.reset_index(drop=True)

In [7]:
timestamps = data.timestamp.to_list()
docs = data.corrected_twit.to_list()
time_list=[]
for timestamp in timestamps:
    time_list.append(timestamp[:7])

In [8]:
okt = Okt()

In [9]:
okt_list = []
for d in tqdm(docs):
    #d = re.sub('[^가-힣]', '', d)
    hangul = re.compile('[^ 0-9가-힣+]')
    sent = hangul.sub(' ', d)
    sent = " ".join(sent.split())
    
    temp = okt.pos(d, stem = True)
    temp = [word[0] for word in temp if (word[1] =='Noun' or word[1] =='Adjective') ]
    okt_list.append(temp)

100%|██████████████████████████████████████████████████████████| 197515/197515 [12:34<00:00, 261.71it/s]


In [10]:
data['okt'] = okt_list

In [11]:
data = data[['twit_id','date','time','twit','refined_twit','corrected_twit','okt']]

In [12]:
vocab = set([])
for i in range(len(data)):
    vocab = vocab.union(set(data.okt[i]))
vocab = list(vocab)

In [14]:
new_stop_words = []
for i in tqdm(range(len(vocab))):
    try: 
        t = get_standard_dictionary(vocab[i])
        key = 0
        for temp in t.json().get('channel').get('item'):
            if temp.get('pos') == '명사' or temp.get('pos') == '형용사':
                key =1
        if key==0 : new_stop_words.append(vocab[i])        
    
    except: 
        new_stop_words.append(vocab[i])

100%|███████████████████████████████████████████████████████████| 33754/33754 [1:20:09<00:00,  7.02it/s]


In [15]:
user_dict_file = "./RawDatasets/txt/user_dic.txt"
user_dict =  [line.strip() for line in open(user_dict_file, encoding="utf-8").readlines()]


In [18]:
f = open("./RawDatasets/txt/new_stopwords.txt",'w')
for word in new_stop_words:
    if word not in user_dict:
        f.write(word+"\n")
    

In [19]:
# stop words removal

In [20]:
stop_word_file = "./RawDatasets/txt/new_stopwords.txt"
stop_word_file2 = "./RawDatasets/txt/stopwords.txt"
stop_words1 =  [line.strip() for line in open(stop_word_file, encoding="utf-8").readlines()]
stop_words2 =  [line.strip() for line in open(stop_word_file2, encoding="utf-8").readlines()]
stop_words = stop_words1+stop_words2

In [21]:
for i in range(len(data)):
    temp = data['okt'][i]
    data['okt'][i] = [word for word in temp if (not word in stop_words and len(word) > 1)]

index = []
for i in range(len(data)):
    if len(data.okt[i]) < 2:
        index.append(i)
data = data.drop(index).reset_index(drop=True)

In [24]:
data

,twit_id,date,time,twit,refined_twit,corrected_twit,okt
0,1553510754581487616,2022-07-30,22:40:01,백신 4차 노바백스로 맞음맞으면서 부작용 사례 물어보니아직 접수 된 건이 하나도 없...,백신 4차 노바백스로 맞음맞으면서 부작용 사례 물어보니아직 접수 된 건이 하나도 없...,백신 4차 노바백스로 맞으면 맞으면서 부작용 사례 물어보니 아직 접수된 건이 하나도...,"[부작용, 사례, 접수, 부위, 통증, 확실하다, 부작용]"
1,1553293555854061571,2022-07-30,08:16:57,노바백스 후기화이자 보다 안 아픔 근데 팔 아파서 잘 못 움직임 근데 이것도 케바케...,노바백스 후기화이자 보다 안 아픔 근데 팔 아파서 잘 못 움직임 근데 이것도 케바케...,노바백스 후기 화이자보다안 아픔 근데 팔 아파서 잘 못 움직임 근데 이것도 때에 따...,"[아픔, 아프다, 움직임, 할머니, 엄마, 아프다]"
2,1552540061635293184,2022-07-28,06:22:50,"1,2차 화이자3차 노바백스생명의 위협을 느껴서 이제서야 3차 맞음근데 15분 있다...",12차 화이자3차 노바백스생명의 위협을 느껴서 이제서야 3차 맞음근데 15분 있다가...,12차 화이자 3차 노바백스 생명의 위협을 느껴서 이제야 3차 맞음 근데 15분 있...,"[생명, 위협, 가라, 스스로]"
3,1551897969049235461,2022-07-26,11:51:23,백신 3차접종 완료자만 입국후 격리면제 가능인정백신 2차 접종까지 지정 백신: 화이...,백신 3차접종 완료자만 입국후 격리면제 가능인정백신 2차 접종까지 지정 백신 화이자...,백신 3차 접종 완료자만 입국 후 격리 면제 가능 인정 백신 2차 접종까지 지정 백...,"[완료, 자만, 입국, 격리, 면제, 가능, 인정, 지정, 지정, 일본, 확진, 완..."
4,1551731672789315586,2022-07-26,00:50:35,@ahomoo1 1차 2차 바이러스벡터 백신3차 mRNA 백신.4차는 유전자 재조합...,1차 2차 바이러스벡터 백신3차 mrna 백신.4차는 유전자 재조합 백신인 노바백스...,1차 2차 바이러스벡터 백신 3차 mrna 백신. 4차는 유전자 재조합 백신인 노바...,"[바이러스, 벡터, 유전자, 조합]"
...,...,...,...,...,...,...,...
123036,1554031516342456320,2022-08-01,09:09:20,4차 노바백스 접종 감상 : 몹시 아픔맞은 직후에 제일 아픈거 같음..왜지.. 왜 ...,4차 노바백스 접종 감상 몹시 아픔맞은 직후에 제일 아픈거 같음..왜지.. 왜 이렇...,4차 노바백스 접종 감상 몹시 아픔 맞은 직후에 제일 아픈 거 같음…. 왜지.. 왜...,"[감상, 아픔, 직후, 아프다, 아프다, 거지]"
123037,1607586232871321600,2022-12-27,03:56:40,나 뭐냐... 격리해제 3개월 지낫는데 동절기 화이자 어쩌구 되는건가?,나 뭐냐... 격리해제 3개월 지낫는데 동절기 화이자 어쩌구 되는건가?,나 뭐냐... 격리해제 3개월 지났는데 동절기 화이자 어쩌고 되는 건가?,"[격리, 해제, 동절기]"
123038,1590878972480741378,2022-11-11,01:27:58,모더나 맞았다. 아예 오늘 외출을 하지 말라고 하시는데 음 오늘 성수동 그냥 가지말...,모더나 맞았다. 아예 오늘 외출을 하지 말라고 하시는데 음 오늘 성수동 그냥 가지말...,모더나 맞았다. 아예 오늘 외출을 하지 말라고 하시는데 음 오늘 성수동 그냥 가지 ...,"[외출, 타이레놀, 월요일, 아침, 부지런하다]"
123039,1555570004912787456,2022-08-05,15:02:44,더킹 관상 레알 준니 좋아하는영화들인데..더킹은 내 영화인생에 단연코1위 최애영화이...,더킹 관상 레알 준니 좋아하는영화들인데..더킹은 내 영화인생에 단연코1위 최애영화이...,더킹 관상 정말 준비 좋아하는영화들인데…. 더킹은 내 영화 인생에 단연코 1위 최애...,"[더킹, 관상, 준비, 영화, 더킹, 영화, 인생, 단연, 최애, 영화, 관심, 계..."


In [25]:
data.to_feather('./covid19.ftr')